## Example: Single Objective Optimization with Constraints
The following example shows how own constraints can be enforced on the input
variables. This only works when Gurobi is selected as the solution strategy,
 i. e. `acq_optimizer="global"`. Constraints are formulated according to
 [documentation](https://www.gurobi.com/documentation/9.0/refman/py_model_addconstr.html).

In [1]:
from entmoot.optimizer.optimizer import Optimizer
from entmoot.benchmarks import SimpleCat

func = SimpleCat()

ModuleNotFoundError: No module named 'entmoot'

Initialize the search space manually.

In [ ]:
from entmoot.space.space import Space
space = Space(func.get_bounds())

get the core of the gurobi model from helper function 'get_core_gurobi_model'

In [ ]:
from entmoot.optimizer.gurobi_utils import get_core_gurobi_model
core_model = get_core_gurobi_model(space)

ordering of variable indices is dependent on space definition

cont_var_dict contains all continuous variables

In [ ]:
x0 = core_model._cont_var_dict[0]
x1 = core_model._cont_var_dict[1]

cat_var_dict contains all categorical variables

In [ ]:
x2 = core_model._cat_var_dict[2]

define constraints accordingly

In [ ]:
core_model.addConstr(x0 + x1 >= 2)
core_model.addConstr(x1 == 1)
core_model.update()

Define Optimizer object

In [ ]:
opt = Optimizer(func.get_bounds(),
                base_estimator="ENTING",
                n_initial_points=0,
                initial_point_generator="random",
                acq_func="LCB",
                acq_optimizer="global",
                random_state=100,
                acq_func_kwargs=None,
                acq_optimizer_kwargs={
                    "add_model_core": core_model
                },
                model_queue_size=None,
                base_estimator_kwargs={
                    "lgbm_params": {"min_child_samples": 1}
                },
                verbose=True,
                )

add initial points that are feasible

In [ ]:
for x in [(1.1, 1.0, 'mult6'), (1.2, 1.0, 'mult6'), (1.3, 1.0, 'pow2')]:
    opt.tell(x, func(x))

run optimizer for 20 iterations

In [ ]:
res = opt.run(func, n_iter=20)
print(f"-> best solution found {res.fun}")